In [1]:
import functools
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
df = pd.read_csv('database.csv')

In [3]:
#shuffle twice
df = df.sample(frac = 1) 
df = df.sample(frac = 1) 

In [4]:
train_df = df.drop(df.index[342:])

In [5]:
train_df.to_csv('train_data.csv',index=False)

In [6]:
test_df = df.drop(df.index[:342])

In [7]:
test_df.to_csv('test_data.csv',index=False)

In [8]:
target = train_df.drop(train_df.columns[:420],axis =1 )

In [9]:
target = target.astype(dtype = 'bool')

In [10]:
train_df = train_df.drop(train_df.columns[420:],axis =1)

# now we have train dataset, test dataset , target dataset

create your own validation set

In [11]:
X = train_df
y = target

In [12]:
# Import train_test_split from Scikit-Learn
from sklearn.model_selection import train_test_split

# Split them into training and validation using NUM_IMAGES 
X_train, X_val, y_train, y_val = train_test_split(X,
                                                  y, 
                                                  test_size=0.2,
                                                  random_state=42)

len(X_train), len(y_train), len(X_val), len(y_val)

(273, 273, 69, 69)

**used batch size as 3 because of the mismatch errors, hence took gcd of 273 and 69**

In [13]:
X_train.shape,X_val.shape,y_train.shape,y_val.shape

((273, 420), (69, 420), (273, 4), (69, 4))

In [16]:
def data_set_train(x,y):
    data = tf.data.Dataset.from_tensor_slices((x,y))
    data = data.shuffle(buffer_size = len(x))
    data = data.batch(3)
    return data

def data_set_val(x,y):
    data = tf.data.Dataset.from_tensor_slices((x,y))
    data = data.batch(3)
    return data

In [17]:
train_data = data_set_train(X_train,y_train)

In [18]:
val_data = data_set_train(X_val,y_val)

In [19]:
train_data,val_data

(<BatchDataset shapes: ((None, 420), (None, 4)), types: (tf.float64, tf.bool)>,
 <BatchDataset shapes: ((None, 420), (None, 4)), types: (tf.float64, tf.bool)>)

# created dataset separately or else creates mismatch...... above

In [20]:
# Check out the different attributes of our data batches
train_data.element_spec, val_data.element_spec

((TensorSpec(shape=(None, 420), dtype=tf.float64, name=None),
  TensorSpec(shape=(None, 4), dtype=tf.bool, name=None)),
 (TensorSpec(shape=(None, 420), dtype=tf.float64, name=None),
  TensorSpec(shape=(None, 4), dtype=tf.bool, name=None)))

In [21]:
INPUT_SHAPE = [None,420]
OUTPUT_SHAPE = len(target.columns)

In [22]:
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

In [23]:
results = np.array([])
np.random.seed(42)
model = Sequential()
model.add(Dense(20, input_dim=420, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(4, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# for train_ix, test_ix in cv.split(X):
#     # prepare data
#     X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
#     y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
#     # fit model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                8420      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 84        
Total params: 8,504
Trainable params: 8,504
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
model.fit(x = train_data , validation_data = val_data ,verbose=0, epochs=100)

In [25]:
predictions = model.predict(val_data, verbose=1)

23/23 [==============================] - 0s 1ms/step


In [26]:
predictions

array([[0.00000000e+00, 1.08958992e-35, 1.00000000e+00, 1.29133919e-23],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.08582698e-36, 1.00000000e+00, 7.22513159e-23],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.47871356e-21, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 9.25955147e-28, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.10007935e-35, 1.00000000e+00, 1.48451967e-24],
       [2.41318732e-20, 1.00000000e+00, 0.00000000e+00, 6.38124775e-10],
       [0.00000000e+00, 2.19181569e-35, 1.00000000e+00, 9.79409310e-24],
       [0.00000000e+00, 1.00000000e+00, 1.24999835e

In [27]:
predictions.shape

(69, 4)

In [29]:
# First prediction
print(predictions[0])
print(f"Max value (probability of prediction): {np.max(predictions[0])}") # the max probability value predicted by the model
print(f"Sum: {np.sum(predictions[0])}") # because we used softmax activation in our model, this will be close to 1
print(f"Max index: {np.argmax(predictions[0])}") # the index of where the max value in predictions[0] occurs
print(f"Predicted label: {target.columns[np.argmax(predictions[0])]}") # the predicted label

[0.0000000e+00 1.0895899e-35 1.0000000e+00 1.2913392e-23]
Max value (probability of prediction): 1.0
Sum: 1.0
Max index: 2
Predicted label: wallfan_corrected


In [41]:
# Create a function to unbatch a batched dataset
def unbatchify(data):
    
    """
    Takes a batched dataset ) Tensors and returns separate arrays
    of data and labels.
    """
    arr= []
    labels = []
  # Loop through unbatched data
    for value, label in data.unbatch().as_numpy_iterator():
        arr.append(value)
        labels.append(target.columns[np.argmax(label)])
    return arr, labels

AttributeError: 'list' object has no attribute 'unbatch'

In [40]:
# Unbatchify the validation data
val_data, val_labels = unbatchify(val_data)
val_data[0], val_labels[0]

AttributeError: 'list' object has no attribute 'unbatch'